# Working with Binance API
- https://python-binance.readthedocs.io/en/latest/overview.html

In [ ]:
import datetime as dt
import os
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import dateparser

import math
import matplotlib.pyplot as plt

from binance.client import Client

# making a websocket system that writes to mongo

In [ ]:
import mytrading

In [ ]:
API_PUBLIC = os.environ.get("B_PUBLIC_KEY")
API_SECRET = os.environ.get("B_SECRET_KEY")
binance_client = Client(API_PUBLIC, API_SECRET)

In [ ]:
import datetime as dt
from mongoengine import *

class DataPoint(Document):
    meta = {
        "collection": "datapoints",
        "indexes": ["timestamp"]
    }
    timestamp = DateTimeField(required=True)

In [ ]:
from binance.websockets import BinanceSocketManager

def open_websocket(client):
    bm = BinanceSocketManager(client)
    
def get_depth_stream_str(symbol, depth=20, update_interval="100ms"):
    return f"{symbol.lower()}@depth@{depth}@{update_interval}"

def process_message(msg):
    print(f"message type: {msg['e']}")
    print(msg)
    
def process_depth_cache(depth_cache):
    print("symbol {}".format(depth_cache.symbol))
    print("top 5 bids")
    print(depth_cache.get_bids()[:5])
    print("top 5 asks")
    print(depth_cache.get_asks()[:5])
    print("last update time {}".format(depth_cache.update_time))

In [ ]:
eth_sym = "ETHBTC"
stream_str = get_depth_stream_str(eth_sym)
bm = BinanceSocketManager(binance_client)
conn_key = bm.start_multiplex_socket([stream_str], process_message)
conn_key

In [ ]:
from binance.depthcache import DepthCacheManager
dcm1 = DepthCacheManager(binance_client, 'BNBBTC', callback=process_depth_cache, bm=bm, refresh_interval=0.1)

In [ ]:
dcm1.close(close_socket=True)

In [ ]:
bm.start()

In [ ]:
bm.stop_socket(conn_key)
bm.close()

In [ ]:
API_PUBLIC = os.environ.get("B_PUBLIC_KEY")
API_SECRET = os.environ.get("B_SECRET_KEY")
binance_client = Client(API_PUBLIC, API_SECRET)



In [ ]:
API_PUBLIC

In [ ]:
bm = BinanceSocketManager(client)
conn_key = bm.start_trade_socket("BNBBTC", process_message)

In [ ]:
ltc_sym = "LTCBTC"
eth_sym = "ETHBTC"
interval = Client.KLINE_INTERVAL_5MINUTE
timedelta = dt.timedelta(days=5)
eth_df = get_closes(eth_sym, interval, timedelta)
ltc_df = get_closes(ltc_sym, interval, timedelta)

eth_lr = get_log_returns(eth_df["c"].values)
ltc_lr = get_log_returns(ltc_df["c"].values)
diff = eth_lr - ltc_lr
hedge_ratio = (eth_df["c"]/ltc_df["c"]).values[1:]

data = np.stack([eth_lr, ltc_lr, diff, hedge_ratio]).T
cols = ["eth", "ltc", "diff", "hedge_ratio"]
ix = eth_df["t1"].values[1:]
df = pd.DataFrame(data=data, columns=cols, index=ix)

In [ ]:
plt.style.use("fivethirtyeight")

In [ ]:
t1 = 10
t2 = 50
fig, ax1 = plt.subplots(1, 1, sharex=True, figsize=(12,6))
linewidth = 3
alpha = 1
ax1.plot(df.iloc[t1:t2]["eth"],linewidth=linewidth, alpha=alpha, label="eth")
ax1.plot(df.iloc[t1:t2]["ltc"], linewidth=linewidth, alpha=alpha, label="ltc")
ax1.plot(df.iloc[t1:t2]["diff"], "--g", linewidth=linewidth, alpha=alpha, label="eth - ltc")
ax1.legend(loc="best")

# using pytorch forecasting

In [ ]:
# starting with no covariates
zero_data = data

In [ ]:
duration = zero_data.size(-2)

prediction = torch.zeros(zero_data.size(0)).unsqueeze(-1)

init_dist = dist.Normal(0, 10).expand([1]).to_event(1)

time
timescale = pyro.sample("timescale", dist.LogNormal(math.log(24), 1))
# Note timescale is a scalar but we need a 1x1 transition matrix (hidden_dim=1),
# thus we unsqueeze twice using [..., None, None].
trans_matrix = torch.exp(-1 / timescale)[..., None, None]
trans_scale = pyro.sample("trans_scale", dist.LogNormal(-0.5 * math.log(24), 1))
trans_dist = dist.Normal(0, trans_scale.unsqueeze(-1)).to_event(1)

# Note the obs_matrix has shape hidden_dim x obs_dim = 1 x 1.
obs_matrix = torch.tensor([[1.]])
obs_scale = pyro.sample("obs_scale", dist.LogNormal(-2, 1))
obs_dist = dist.Normal(0, obs_scale.unsqueeze(-1)).to_event(1)

noise_dist = dist.GaussianHMM(
    init_dist, trans_matrix, trans_dist, obs_matrix, obs_dist, duration=duration)

In [ ]:
prediction.shape

In [ ]:
timesclae

In [ ]:
class Model1(ForecastingModel):
    def model(self, zero_data, covariates):
        duration = zero_data.size(-2)
        
        prediction = torch.zeros(zero_data.size(0)).unsqueeze(-1)

        init_dist = dist.Normal(0, 10).expand([1]).to_event(1)

        timescale = pyro.sample("timescale", dist.LogNormal(math.log(24), 1))
        # Note timescale is a scalar but we need a 1x1 transition matrix (hidden_dim=1),
        # thus we unsqueeze twice using [..., None, None].
        trans_matrix = torch.exp(-1 / timescale)[..., None, None]
        trans_scale = pyro.sample("trans_scale", dist.LogNormal(-0.5 * math.log(24), 1))
        trans_dist = dist.Normal(0, trans_scale.unsqueeze(-1)).to_event(1)

        # Note the obs_matrix has shape hidden_dim x obs_dim = 1 x 1.
        obs_matrix = torch.tensor([[1.]])
        obs_scale = pyro.sample("obs_scale", dist.LogNormal(-2, 1))
        obs_dist = dist.Normal(0, obs_scale.unsqueeze(-1)).to_event(1)

        noise_dist = dist.GaussianHMM(
            init_dist, trans_matrix, trans_dist, obs_matrix, obs_dist, duration=duration)
        self.predict(noise_dist, prediction)

In [ ]:
data = torch.from_numpy(df["eth"].values).view(-1, 1)
covariates = torch.zeros(data.size(0), 0)
model = Model1()
forecaster = Forecaster(model, data, covariates)

In [ ]:
for name, value in forecaster.guide.median().items():
    if value.numel() == 1:
        print("{} = {:0.4g}".format(name, value.item()))